In [1]:
import os
import cv2
import shutil
import PIL
import random
from pathlib import Path
from PIL import Image
from tqdm import tqdm
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import classification_report,confusion_matrix
import numpy as np
import matplotlib.pyplot as plt

In [7]:
CLASS_NAMES = ['correctly-masked', 'incorrectly-masked', 'not-masked']
BASE_DIR = '../'
MODEL_DIR = BASE_DIR+'model/'
TEST_DIR = BASE_DIR+'test/'
IMAGES_DIR = TEST_DIR+'images/test-images/'
PREDICTIONS_DIR = TEST_DIR+'predictions/'
IMG_HEIGHT = 256
IMG_WIDTH = 256

In [6]:
model = keras.models.load_model(MODEL_DIR)

In [8]:
predict_dir = IMAGES_DIR
predict_ds = tf.keras.preprocessing.image_dataset_from_directory(
    predict_dir,
    image_size = (IMG_HEIGHT, IMG_WIDTH),
    seed = 123)
x_pred = []
for image, label in tqdm(predict_ds.unbatch().take(-1)):
    img = image.numpy().astype("uint8")
    x_pred.append(img)

x_pred = np.array(x_pred)

predictions = np.argmax(model.predict(x_pred), axis=-1)
predictions = predictions.reshape(1,-1)[0]

1it [00:00,  7.95it/s]

Found 1368 files belonging to 1 classes.


1368it [00:01, 1210.18it/s]


In [9]:
[Path(PREDICTIONS_DIR+clazz).mkdir(parents=True, exist_ok=True) for clazz in CLASS_NAMES]
for i in range(len(predictions)):
    img = Image.fromarray(x_pred[i])
    dest_dir = PREDICTIONS_DIR+CLASS_NAMES[predictions[i]] + '/'
    fname = dest_dir + str(i) + '.jpeg'
    img.save(fname)